In [ ]:
# Pandamart 

# import
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from random import shuffle, randint
import win32com.client
import time

# time
start_time = time.time()

# particulars
brands = ['Rin', 'Wheel', 'Surf Excel']
monitor = ['Rin Advanced Synthetic Laundry Detergent Powder 1kg', 'Rin Advanced Synthetic Laundry Detergent Powder 2kg', 'Rin Advanced Synthetic Laundry Detergent Powder 500g', 'Rin Liquid Detergent 400ml', 'Rin Liquid Detergent 800ml', 'Surf Excel Synthetic Laundry Detergent Powder 200g', 'Surf Excel Synthetic Laundry Detergent Powder 500g', 'Surf Excel Synthetic Laundry Detergent Powder 1kg', 'Surf Excel Matic Liquid Detergent Top Load I 500 ml', 'Surf Excel Matic Liquid Detergent Top Load 1000ml', 'Wheel Washing (Detergent) Powder 2in1 Clean & Fresh 200g', 'Wheel Washing (Detergent) Powder 2in1 Clean & Fresh 1Kg', 'Wheel Washing (Detergent) Powder 2in1 Clean & Fresh 500g', 'Wheel Washing (Detergent) Powder 2in1 Clean & Fresh 2Kg', 'Wheel Washing Laundry Bar 125g', '(Buy & Get Free) Buy Surf Excel Washing Powder With Aloe Vera 1kg & Get 1pc Bucket Free', '(Buy & Get Free) Buy Rin Washing Powder Power Bright 1kg & Get Container Free', '(Buy & Get Free) Buy 1pc Rin Advanced Synthetic Laundry Detergent Powder 2kg & Get 1pc Bucket Free', 'Rin Washing Liquid 800ml ( Buy 2 get 20% Off )', 'Rin Washing Liquid 400Ml buy 2 get 15% off']

# urls
urls = [
    'https://www.foodpanda.com.bd/darkstore/w2lx/pandamart-gulshan-w2lx',
    'https://www.foodpanda.com.bd/darkstore/h5rj/pandamart-bashundhara',
    'https://www.foodpanda.com.bd/darkstore/ta7z/pandamart-dhanmondi',
    'https://www.foodpanda.com.bd/darkstore/n7ph/pandamart-uttara',
    'https://www.foodpanda.com.bd/darkstore/v1ts/pandamart-mogbazar',
    'https://www.foodpanda.com.bd/darkstore/q4hz/pandamart-sylhet-02',
    'https://www.foodpanda.com.bd/darkstore/a2er/pandamart-khulna',
    'https://www.foodpanda.com.bd/darkstore/w2nv/pandamart-chittagong-1'
]
shuffle(urls)
len_urls = len(urls)

# expressions, ref: https://www.w3schools.com/charsets/ref_emoji.asp
emos = [9888, 9940, 9889, 128128, 127875, 128030, 128064, 128073
    # exclaim, minus, thunder, skull, pumpkim, bug, look, indicate
]

# preference
options = webdriver.ChromeOptions()
options.add_argument("ignore-certificate-errors")

# delay
for i in range(len_urls):
    if i > 0: time.sleep(randint(60, 70))
        
    # open window
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    
    # url
    print("Monitoring: " + urls[i])
    driver.get(urls[i])
    
    # accumulator
    sku = []
    
    # brand
    for b in brands:
        print("Scraping for: " + b)
        elem = driver.find_element(By.XPATH, '//*[@id="groceries-menu-react-root"]/div/div/div[2]/div/section/div[3]/div/div/div/div/div[1]/input')
        elem.send_keys(b + "\n")

        # scroll
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height: break
            last_height = new_height

        # soup
        soup_init = BeautifulSoup(driver.page_source, 'html.parser')
        soup = soup_init.find_all('div', attrs={'class', 'box-flex product-card-attributes'})

        # scrape
        for s in soup:
            try: val = s.find('p', attrs={'class', 'product-card-name'}).get_text()
            except: val = None
            sku.append(val)

        # back
        driver.back()
        
    # email
    ol = win32com.client.Dispatch("outlook.application")
    olmailitem = 0x0
    newmail = ol.CreateItem(olmailitem)

    # report
    oos = ""
    for m in monitor: 
        if m not in sku: oos = oos + "<br>&nbsp;&nbsp;&nbsp;• " + m
    oos = "&#" + str(emos[i]) + " <b>" + str(oos.count("• ")) + "</b> fabsol SKUs missing from URL location: <i>" + urls[i].split("/")[-1] + "</i>" + oos if oos.count("• ")>0 else ""

    # Teams
    newmail.Subject = "Pandamart UBL OOS"
    newmail.To = "Eagle Eye - Alerts <93f21d6e.Unilever.onmicrosoft.com@emea.teams.ms>; ashish.ecomexclusive@gmail.com"
    newmail.HTMLbody = oos + "<br><br>"
    if len(oos) > 0: newmail.Send()
        
    # close window
    driver.close()

# stats
elapsed_time = time.time() - start_time
print("\nElapsed time to report (mins): " + str(round(elapsed_time / 60.00, 2)))